In [1]:
import sys
sys.path.append(r'C:/Program Files (zk)/PythonFiles/AClassification/SoundDL-CoughVID')
import os
import yaml
import time
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchaudio
from models.conv_vae import ConvVAE, vae_loss
from readers.coughvid_reader import CoughVID_Class, CoughVID_Dataset
from readers.featurizer import Wave2Mel
from pretrained.wav2vec import Wav2Vec
from readers.collate_fn import collate_fn

In [11]:
encoder = Wav2Vec(pretrained=True)

In [2]:
import pandas as pd
src_data = pd.read_csv("./datasets/waveinfo_labedfine_forcls.csv", header=0, index_col=0, delimiter=',')
print("原始数据：", src_data.shape)
print(src_data.iloc[:, [0, 6]].groupby("status_full").count())

原始数据： (6341, 7)
             filename
status_full          
0                2114
1                3288
2                 939


In [ ]:
from torch.utils.data import DataLoader
train_x, train_y, test_x, test_y = CoughVID_Class(isdemo=True)

tic = time.time()
cough_dataset = CoughVID_Dataset(train_x, train_y)
toc = time.time()
print("Train Dataset Creat Completely, cost time:", toc-tic)

tic = time.time()
valid_dataset = CoughVID_Dataset(path_list=test_x, label_list=test_y)
toc = time.time()
print("Valid Dataset Creat Completely, cost time:", toc-tic)

In [4]:
configs = {
    "run_save_dir": "./runs/wav2vec_coughvid/",
    "model":{
        "num_class": 3,
        "input_length": 94,
        "wav_length": 48000,
        "input_dim": 512,
        "n_mels": 128,
        },
    "fit":{
        "batch_size": 64,
        "epochs" : 23,
        "start_scheduler_epoch": 6
        },
}

device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
num_epoch = configs["fit"]["epochs"]
# klw = 0.00025
# istrain: 如果是评估环节，设为False，读取测试集，并且不创建optimizer
# isdemo: 如果只是测试一下，设为True，仅读取32条数据方便快速测试是否有bug
# istrain, isdemo = True, False

In [5]:
class LSTM_Classifier(nn.Module):
    def __init__(self, inp_size, hidden_size, n_classes):
        super(LSTM_Classifier, self).__init__()
        self.lstm = nn.LSTM(inp_size, hidden_size, batch_first=True)
        self.classifier = nn.Linear(hidden_size, n_classes)

    def forward(self, x):
        lstm_out, (hidden, _) = self.lstm(x.transpose(1,2))
        lstm_out = lstm_out[:, -1, :]
        out = self.classifier(lstm_out)
        return out
        
class LSTM_Attn_Classifier(nn.Module):
    def __init__(self, inp_size, hidden_size, n_classes, return_attn_weights=False, attn_type='dot'):
        super(LSTM_Attn_Classifier, self).__init__()
        self.return_attn_weights = return_attn_weights
        self.lstm = nn.LSTM(inp_size, hidden_size, batch_first=True)
        self.attn_type = attn_type

        if self.attn_type == 'dot':
            self.attention = DotAttention()
        elif self.attn_type == 'soft':
            self.attention = SoftAttention(hidden_size, hidden_size)

        self.classifier = nn.Linear(hidden_size, n_classes)


    def forward(self, x):
        lstm_out, (hidden, _) = self.lstm(x.transpose(1,2))

        if self.attn_type == 'dot':
            attn_output = self.attention(lstm_out, hidden)
            attn_weights = self.attention._get_weights(lstm_out, hidden)
        elif self.attn_type == 'soft':
            attn_output = self.attention(lstm_out)
            attn_weights = self.attention._get_weights(lstm_out)

        out = self.classifier(attn_output)
        if self.return_attn_weights:
            return out, attn_weights
        else:
            return out

In [16]:

# model loss_function optimizer scheduler
print("All model and loss are on device:", device)
model = LSTM_Classifier(inp_size=298, hidden_size=64, n_classes=3).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=1e-4, max_lr=1e-1, step_size_up=10)
print("Create TDNN, Adam with lr=1e-3, CosineAnnealingLR Shceduler")


All model and loss are on device: cuda
Create TDNN, Adam with lr=1e-3, CosineAnnealingLR Shceduler


In [6]:

criterion=nn.CrossEntropyLoss().to(device)
# print("Create VAELoss...")

train_loader = DataLoader(cough_dataset, batch_size=32, shuffle=False,
                          collate_fn=collate_fn)
test_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False,
                          collate_fn=collate_fn)
for i, (x_wav, y_label, max_len_rate) in enumerate(train_loader):
    print(x_wav.shape)
    print(y_label)
    print(max_len_rate)
    x_mel = w2m(x_wav)
    print(x_mel[0])
    break
print("Create Training Loader and Valid Loader.")

# w2m = Wave2Mel(sr=16000)

All model and loss are on device: cuda
Create TDNN, Adam with lr=1e-3, CosineAnnealingLR Shceduler


NameError: name 'train_dataset' is not defined

In [9]:
timestr = time.strftime("%Y%m%d%H%M", time.localtime())
run_save_dir = configs["run_save_dir"] + timestr + f'_tdnn_focalloss/'
os.makedirs(run_save_dir, exist_ok=True)
print("创建运行保存文件", run_save_dir)

创建运行保存文件 ./runs/wav2vec_coughvid/202404291803_tdnn_focalloss/


In [17]:
history1 = []
for epoch_id in range(configs["fit"]["epochs"]):
    # ---------------------------
    # -----------TRAIN-----------
    # ---------------------------
    model.train()
    for x_idx, (x_wav, y_label, _) in enumerate(tqdm(train_loader, desc="Training")):
        x_mel = encoder(x_wav).transpose(1,2).to(device)
        y_label = torch.tensor(y_label, device=device)
        # print("shape of x_mel:", x_mel.shape)
        optimizer.zero_grad()
        y_hat = model(x_mel)
        pred_loss = criterion(y_hat, y_label)
        pred_loss.backward()
        optimizer.step()

        if x_idx > 2:
            history1.append(pred_loss.item())
        if x_idx % 60 == 0:
            print(f"Epoch[{epoch_id}], mtid pred loss:{pred_loss.item():.4f}")
    if epoch_id >= configs["fit"]["start_scheduler_epoch"]:
        scheduler.step()

    # ---------------------------
    # -----------SAVE------------
    # ---------------------------
    plt.figure(0)
    plt.plot(range(len(history1)), history1, c="green", alpha=0.7)
    plt.savefig(run_save_dir + f'cls_loss_iter_{epoch_id}.png')
    plt.close()
    # if epoch > 6 and epoch % 2 == 0:
    os.makedirs(run_save_dir + f"model_epoch_{epoch_id}/", exist_ok=True)
    tmp_model_path = "{model}model_{epoch}.pth".format(
        model=run_save_dir + f"model_epoch_{epoch_id}/",
        epoch=epoch_id)
    torch.save(model.state_dict(), tmp_model_path)
    # ---------------------------
    # -----------TEST------------
    # ---------------------------
    model.eval()
    heatmap_input = None
    labels = None
    for x_idx, (x_wav, y_label, _) in enumerate(tqdm(valid_loader, desc="Validate")):
        x_mel = encoder(x_wav).transpose(1,2).to(device)
        y_label = torch.tensor(y_label, device=device)
        y_hat = model(x_mel)
        pred_loss = criterion(y_hat, y_label)
        if x_idx == 0:
            heatmap_input, labels = y_pred, y_label
        else:
            heatmap_input = torch.concat((heatmap_input, y_pred), dim=0)
            labels = torch.concat((labels, y_label), dim=0)
        # if x_idx * configs["fit"]["batch_size"] > 800:
        #     break
    print("heatmap_input shape:", heatmap_input.shape)
    print("lables shape:", labels.shape)
    # if epoch > 3:
    #     self.plot_reduction(resume_path="", load_epoch=epoch, reducers=["heatmap"])
    heatmap_input = heatmap_input.detach().cpu().numpy()
    labels = labels.detach().cpu().numpy()
    calc_accuracy(pred_matrix=heatmap_input, label_vec=labels,
                  save_path=run_save_dir + f"/accuracy_epoch_{epoch_id}.txt")
    plot_heatmap(pred_matrix=heatmap_input, label_vec=labels,
                 ticks=["healthy", "symptomatic", "COVID-19"],
                 save_path=run_save_dir + f"/heatmap_epoch_{epoch_id}.png")
print("============== END TRAINING ==============")

Training:   0%|                                                                                | 0/189 [00:00<?, ?it/s]C:\Users\zhaoke\AppData\Local\Temp\ipykernel_2752\1291739228.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_label = torch.tensor(y_label, device=device)
Training:   1%|▍                                                                       | 1/189 [00:06<19:20,  6.17s/it]

Epoch[0], mtid pred loss:1.0374


Training:  32%|██████████████████████▉                                                | 61/189 [05:57<12:21,  5.79s/it]

Epoch[0], mtid pred loss:0.9923


Training:  64%|████████████████████████████████████████████▊                         | 121/189 [11:48<06:39,  5.87s/it]

Epoch[0], mtid pred loss:1.1001


Training:  96%|███████████████████████████████████████████████████████████████████   | 181/189 [17:38<00:46,  5.78s/it]

Epoch[0], mtid pred loss:1.2097


Training: 100%|██████████████████████████████████████████████████████████████████████| 189/189 [18:24<00:00,  5.84s/it]


NameError: name 'valid_loader' is not defined